<a href="https://colab.research.google.com/github/KS9910/Multivariate-Time-Series-Classification/blob/main/GAF_Images_save.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install mne

In [2]:
import mne
from mne.datasets import eegbci
from mne.io import concatenate_raws, read_raw_edf

#Define the parameters
subject = 1  # use data from subject 1
hand_img_runs = [4, 8, 12]  # hand runs
feet_img_runs = [6, 10, 14]  # feet runs
hand_exe_runs = [3, 7, 11]  # hand runs
feet_exe_runs = [5, 9, 13]  # feet runs
hand_img_files = eegbci.load_data(subject, hand_img_runs, '../datasets/')
feet_img_files = eegbci.load_data(subject, feet_img_runs, '../datasets/')
hand_exe_files = eegbci.load_data(subject, hand_exe_runs, '../datasets/')
feet_exe_files = eegbci.load_data(subject, feet_exe_runs, '../datasets/')


In [3]:
#Read raw data files where each file contains a run
hand_img_raws = [read_raw_edf(f, preload=True) for f in hand_img_files]
hand_exe_raws = [read_raw_edf(f, preload=True) for f in hand_exe_files]
feet_img_raws = [read_raw_edf(f, preload=True) for f in feet_img_files]
feet_exe_raws = [read_raw_edf(f, preload=True) for f in feet_exe_files]
#Combine all loaded runs
hand_img_obj = concatenate_raws(hand_img_raws)
hand_exe_obj = concatenate_raws(hand_exe_raws)
feet_img_obj = concatenate_raws(feet_img_raws)
feet_exe_obj = concatenate_raws(feet_exe_raws)

Extracting EDF parameters from /datasets/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /datasets/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /datasets/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /datasets/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /datasets/MNE-eegbci-

In [4]:
import numpy as np
#Extract events from raw data
events_hi, event_ids_hi = mne.events_from_annotations(hand_img_obj, event_id='auto')
events_fi, event_ids_fi = mne.events_from_annotations(feet_img_obj, event_id='auto')
events_hx, event_ids_hx = mne.events_from_annotations(hand_exe_obj, event_id='auto')
events_fx, event_ids_fx = mne.events_from_annotations(feet_exe_obj, event_id='auto')

Used Annotations descriptions: ['T0', 'T1', 'T2']
Used Annotations descriptions: ['T0', 'T1', 'T2']
Used Annotations descriptions: ['T0', 'T1', 'T2']
Used Annotations descriptions: ['T0', 'T1', 'T2']


In [5]:
tmin, tmax = -1, 4  # define epochs around events (in s)
#event_ids = dict(hands=2, feet=3)  # map event IDs to tasks

epochs_hi = mne.Epochs(hand_img_obj, events_hi, event_ids_hi, tmin - 0.5, tmax + 0.5, baseline=None, preload=True)
epochs_fi = mne.Epochs(hand_img_obj, events_fi, event_ids_fi, tmin - 0.5, tmax + 0.5, baseline=None, preload=True)
epochs_hx = mne.Epochs(hand_exe_obj, events_hx, event_ids_hx, tmin - 0.5, tmax + 0.5, baseline=None, preload=True)
epochs_fx = mne.Epochs(hand_exe_obj, events_fx, event_ids_fx, tmin - 0.5, tmax + 0.5, baseline=None, preload=True)

Not setting metadata
90 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 90 events and 961 original time points ...
3 bad epochs dropped
Not setting metadata
90 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 90 events and 961 original time points ...
3 bad epochs dropped
Not setting metadata
90 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 90 events and 961 original time points ...
3 bad epochs dropped
Not setting metadata
90 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 90 events and 961 original time points ...
3 bad epochs dropped


In [6]:
#Access to the data
data1 = epochs_hi._data
data2 = epochs_fi._data
data3 = epochs_hx._data
data4 = epochs_fx._data
# print(data.shape)
# print(data[0,0,:])

In [7]:
# Concatenate along the first axis (axis=0)
resultant_data = np.concatenate((data1, data2, data3, data4), axis=0)

# Check the shape of the resultant matrix
print(f"The shape of the resultant matrix is: {resultant_data.shape}")

The shape of the resultant matrix is: (348, 64, 961)


In [8]:
n_events = len(resultant_data) # or len(epochs.events)
print("Number of events: " + str(n_events))

n_channels = len(resultant_data[0,:]) # or len(epochs.ch_names)
print("Number of channels: " + str(n_channels))

n_times = len(resultant_data[0,0,:]) # or len(epochs.times)
print("Number of time instances: " + str(n_times))

Number of events: 348
Number of channels: 64
Number of time instances: 961


In [9]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def calculate_gadf(X):
    # Normalize input data to range [-1, 1]
    scaler = MinMaxScaler(feature_range=(-1, 1))
    X_normalized = scaler.fit_transform(X)

    # Calculate the GADF for each time series
    n_samples, n_timepoints = X_normalized.shape
    gadf_matrices = []

    for i in range(n_samples):
        X_i = X_normalized[i].reshape(-1, 1)
        theta = np.arccos(np.clip(X_i, -1, 1))
        gadf_matrix = np.sin(theta - theta.T)
        gadf_matrices.append(gadf_matrix)

    # Stack all GADF matrices along a new axis to form a 3D matrix
    gadf_matrices = np.stack(gadf_matrices, axis=0)
    return gadf_matrices


In [10]:
# Preprocess EEG data (flatten)
eeg_data_flat = resultant_data.reshape(-1, n_channels * n_times)
scaler = MinMaxScaler()
eeg_data_normalized = scaler.fit_transform(eeg_data_flat)
print("eeg_data_normalized size:", eeg_data_normalized.shape)
temp=eeg_data_normalized[0].reshape(n_channels, n_times)
print("eeg_data_normalized [0] size:", temp.shape)
eeg_images = []
# Convert EEG data to images
def process_in_batches(eeg_data_normalized, batch_size):
    # n_samples = eeg_data_normalized.shape[0]
    n_samples = 120;

    for start in range(110, n_samples, batch_size):
        end = min(start + batch_size, n_samples)
        batch = eeg_data_normalized[start:end]

        for epoch_data in batch:
            temp = epoch_data.reshape(n_channels, n_times)
            image = calculate_gadf(temp)
            eeg_images.append(image)

        print(f"Processed batch {start//batch_size + 1}/{(n_samples + batch_size - 1) // batch_size}")

    return eeg_images

batch_size = 10  # Adjust batch size according to your memory constraints
eeg_images = process_in_batches(eeg_data_normalized, batch_size)
print(len(eeg_images))

eeg_data_normalized size: (348, 61504)
eeg_data_normalized [0] size: (64, 961)
Processed batch 12/12
10


In [11]:
import tensorflow as tf
from tensorflow.keras import layers, models

input_shape = (964, 964, 1)

# Encoder
encoder_input = layers.Input(shape=input_shape)
x = layers.Conv2D(1, (3, 3), activation='relu', padding='same')(encoder_input)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(1, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

# Decoder
x = layers.Conv2D(1, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(1, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

# Autoencoder Model
autoencoder = models.Model(encoder_input, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# Summary of the autoencoder
autoencoder.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 964, 964, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 964, 964, 1)       10        
                                                                 
 max_pooling2d (MaxPooling2  (None, 482, 482, 1)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 482, 482, 1)       10        
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 241, 241, 1)       0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 241, 241, 1)       10    

In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models

def rnn_attention(input_tensor):
    # Reshape input tensor to be compatible with RNN layer
    reshaped_tensor = layers.Reshape((input_tensor.shape[1], input_tensor.shape[2] * input_tensor.shape[3]))(input_tensor)

    # Apply GRU layer with return_sequences=True to get hidden states for each time step
    gru_output = layers.GRU(64, return_sequences=True)(reshaped_tensor)

    # Compute attention scores for each time step
    attention_scores = layers.Dense(1, activation='sigmoid')(gru_output)

    # Normalize attention scores using softmax
    attention_weights = layers.Softmax()(attention_scores)

    # Weighted sum of hidden states with attention weights
    attended_features = layers.Multiply()([gru_output, attention_weights])

    # Reshape to a 2D representation for further processing
    attended_features = layers.Reshape((input_tensor.shape[1], 64))(attended_features) # Reshape to (256, 64)

    return attended_features

# Example usage:
input_shape = (241, 241, 1)  # Shape of the grayscale GAF image
input_tensor = layers.Input(shape=input_shape)

# Apply RNN attention mechanism
attended_features = rnn_attention(input_tensor)

# You might need additional layers here to process the attended_features
# before reshaping it back to the original image dimensions if needed.

# Create Keras model to use the RNN attention layer
model = models.Model(inputs=input_tensor, outputs=attended_features)

In [13]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

eeg_images_batch = []
for i in range(0,5):
  # Assuming eeg_images is already prepared in the shape (348, 64, 64)
  eeg_images_1 = np.array(eeg_images[i])
  # Add padding to the second and third dimensions
  pad_width = ((0, 0), (1, 2), (1, 2))
  eeg_images_1_padded = np.pad(eeg_images_1, pad_width, mode='constant', constant_values=0)

  # Add a new dimension at the end to change shape from (64, 964, 964) to (64, 964, 964, 1)
  eeg_images_1_padded = np.expand_dims(eeg_images_1_padded, axis=-1)

  # Train the autoencoder
  autoencoder.fit(eeg_images_1_padded, eeg_images_1_padded, epochs=4, batch_size=1, shuffle=True)

  # Extract the encoder model
  encoder = models.Model(encoder_input, encoded)
  encoded_images = encoder.predict(eeg_images_1_padded)
  print("Shape of encoded images:", encoded_images.shape)

  # RNN Attention for enhancing the features of GAF images
  after_attention_image = model.predict(encoded_images)
  print("Shape of result:", after_attention_image.shape)  # Output: (64, 256, 64)

  eeg_images_batch.append(after_attention_image)

# Convert the list eeg_images_batch to a NumPy array
eeg_images_batch_array = np.array(eeg_images_batch)

# saving into the drive
from google.colab import drive
drive.mount('/content/gdrive')

with open('/content/gdrive/My Drive/GAF_image_s1/gaf_images_e_110-115.npy', 'wb') as f:
  np.save(f, eeg_images_batch_array)

Epoch 1/4
64/64 [==============================] - 6s 9ms/step - loss: 0.5698
Epoch 2/4
64/64 [==============================] - 0s 7ms/step - loss: 0.4377
Epoch 3/4
64/64 [==============================] - 0s 7ms/step - loss: 0.3003
Epoch 4/4
2/2 [==============================] - 1s 40ms/step
Shape of encoded images: (64, 241, 241, 1)
2/2 [==============================] - 0s 10ms/step
Shape of result: (64, 241, 64)
Epoch 1/4
64/64 [==============================] - 1s 8ms/step - loss: 0.2707
Epoch 2/4
64/64 [==============================] - 0s 7ms/step - loss: 0.2632
Epoch 3/4
64/64 [==============================] - 0s 7ms/step - loss: 0.2572
Epoch 4/4
2/2 [==============================] - 0s 42ms/step
Shape of encoded images: (64, 241, 241, 1)
2/2 [==============================] - 0s 11ms/step
Shape of result: (64, 241, 64)
Epoch 1/4
64/64 [==============================] - 1s 9ms/step - loss: 0.2402
Epoch 2/4
64/64 [==============================] - 0s 7ms/step - loss: 0.2389


In [14]:

# Now you can access the shape
print(eeg_images_batch_array.shape)

(5, 64, 241, 64)
